In [1]:
import os
from azureml.core import Workspace, Datastore, Dataset
from azureml.pipeline.wrapper import Module, dsl, Pipeline
from azureml.pipeline.wrapper._dataset import get_global_dataset_by_path
from azureml.core.compute import AmlCompute, ComputeTarget

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

aml_compute_target = "aml-compute"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except:
    print("Creating new compute target: {}".format(aml_compute_target))

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                min_nodes=1,
                                                                max_nodes=4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
DesignerTest-EUS
DesignerTestRG
eastus
4faaaf21-663f-4391-96fd-47197c630979
Found existing compute target: aml-compute


In [3]:
blob_input_data = get_global_dataset_by_path(
    ws, 'Automobile_price_data',
    'GenericCSV/Automobile_price_data_(Raw)')
notebook_module_func = Module.from_notebook(ws, 'authoring/sample_module_from_notebook/from_notebook.ipynb')
module1 = notebook_module_func(input=blob_input_data, param1='aaa0', param2='bbb0',i=1,j=2,flag=False)
module2 = notebook_module_func(input=module1.outputs['output'], param1='aaa1', param2='bbb1',i=2,j=3,flag=True)
steps = [module1, module2]

INFO     - Extracting dsl module info from the notebook
INFO     - Module entry file '_from_notebook.py' is dumped.


In [4]:
pipeline_name = "notebook_test"
pipeline = Pipeline(name=pipeline_name, nodes=steps,
                    outputs={**steps[-1].outputs})
run = pipeline.submit(pipeline_name, "aml-compute")
print(run.id)
run.wait_for_completion()



Submitted PipelineRun c06c490e-310d-460c-a430-46892d441c9f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/notebook_test/runs/c06c490e-310d-460c-a430-46892d441c9f?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS
c06c490e-310d-460c-a430-46892d441c9f
PipelineRunId: c06c490e-310d-460c-a430-46892d441c9f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/notebook_test/runs/c06c490e-310d-460c-a430-46892d441c9f?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS


<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_ee4c2ee3-a507-4b1e-9dd2-d66001151f70_widget', env_json='{}', graph_jso…

<RunStatus.completed: 'Completed'>

In [7]:
pipeline.run()

<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_2b019728-9324-4be6-828c-3f10bfe0ea58_widget', env_json='{"subscription…

Working dir: C:\Users\zhohuang\AppData\Local\Temp\notebook_test\80f7bb71-98f3-482c-b25d-0eca4ffe1108
RunId: 80f7bb71-98f3-482c-b25d-0eca4ffe1108
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/notebook_test/runs/80f7bb71-98f3-482c-b25d-0eca4ffe1108?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS
Preparing pipeline dataset.
Downloaded GenericCSV/Automobile_price_data_(Raw)/_data.parquet, 1 files out of an estimated total of 5
Downloaded GenericCSV/Automobile_price_data_(Raw)/_meta.yaml, 2 files out of an estimated total of 5
Downloaded GenericCSV/Automobile_price_data_(Raw)/data_type.json, 3 files out of an estimated total of 5
Downloaded GenericCSV/Automobile_price_data_(Raw)/schema/_schema.json, 4 files out of an estimated total of 5
Downloaded GenericCSV/Automobile_price_data_(Raw)/_data.visualization, 5 files out of an estimated total of 5
Prepared pipeline dataset.


'Completed'

You can also create module from notebook file by using Azure CLI

In [2]:
!az ml module init --source authoring/sample_module_from_notebook/from_notebook.ipynb

INFO     - Module project builder version: 0.1.0.19749058 Python executable: C:\Users\zhohuang\Miniconda3\envs\aml\python.exe
INFO     ========== Init started: c:\Users\zhohuang\Desktop\hello-aml-modules\samples_pipelines\how-to-use ==========
INFO     - Determining if source authoring/sample_module_from_notebook/from_notebook.ipynb is function...
INFO     - Determining if source authoring/sample_module_from_notebook/from_notebook.ipynb is notebook...
INFO     - Attempting to generate dsl.module from notebook...
INFO     - Extracting dsl module info from the notebook
INFO     - Module entry file 'authoring\sample_module_from_notebook\from_notebook_entry.py' is dumped.
INFO     - Initializing from notebook...
INFO     - 		Created 	 authoring\sample_module_from_notebook\from_notebook_entry.py -> authoring\sample_module_from_notebook\from_notebook_entry.spec.yaml
INFO     - 		Created 	 from_notebook_entry_test.ipynb
INFO     - 		Created 	 from_notebook_entry_test.py
INFO     - 		Created 	